In [17]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
import math
import numpy as np
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
def deg_to_rad(dr):
    return (dr*math.pi)/180


In [18]:
df = pd.read_csv('train.csv')
df['label'] = df['label'].map({'correct': 1, 'incorrect': 0})
df = df.dropna()
#df = df.drop(['drop_time'], axis = 1)

df['pick_lat'] = deg_to_rad(df['pick_lat'])
df['pick_lon'] = deg_to_rad(df['pick_lon'])
df['drop_lat'] = deg_to_rad(df['drop_lat'])
df['drop_lon'] = deg_to_rad(df['drop_lon'])
dlon = df['drop_lon'] - df['pick_lon']
dlat = df['drop_lat'] - df['pick_lat']
df['dist'] = np.sin(dlat/2)**2 + np.cos(df['pick_lat'])*np.cos(df['drop_lat'])*np.sin(dlon/2)**2
df['dist'] = 2*np.arctan2(df['dist']**0.5,(1-df['dist'])**0.5)
R = 6373.0
df['dist'] = R*df['dist']

In [19]:
df['pickup_time'] = [float((d.split(' ')[1]).split(':')[0]+'.'+(d.split(' ')[1]).split(':')[1]) for d in df['pickup_time']]
df['drop_time'] = [float((d.split(' ')[1]).split(':')[0]+'.'+(d.split(' ')[1]).split(':')[1]) for d in df['drop_time']]

In [20]:
df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,dist
0,189123628,10.5,834.0,56.0,0.0000,64.0,0.20,0.34,0.119774,1.394506,0.120485,1.394139,270.32,1,5.094369
1,189125358,10.5,791.0,47.0,0.0000,134.0,0.56,1.09,0.120181,1.394490,0.120667,1.394384,197.85,1,3.169052
2,189125719,10.5,1087.0,80.0,0.0000,61.0,1.08,1.26,0.120574,1.393909,0.121068,1.394773,301.64,1,6.307375
3,189127273,10.5,598.0,271.0,15.6638,68.0,2.27,2.37,0.120876,1.394335,0.120907,1.394467,82.30,1,0.862217
5,189129552,10.5,3407.0,182.0,0.0000,112.0,5.38,6.35,0.124512,1.394464,0.120753,1.393905,1065.02,1,24.214638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,22.07,22.21,0.127247,1.407376,0.127215,1.407708,198.26,1,2.106037
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,23.07,23.43,0.120527,1.393673,0.121316,1.395197,581.23,1,10.871789
17173,213813930,10.5,263.0,9.0,0.0000,110.0,23.21,23.25,0.123780,1.394518,0.123942,1.394548,76.20,1,1.045847
17174,213815405,10.5,858.0,115.0,0.0000,317.0,23.39,23.53,0.121220,1.394113,0.121051,1.394536,133.31,1,2.879981


In [21]:
df = df.drop(['drop_time','meter_waiting_fare','meter_waiting_till_pickup','tripid','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

In [22]:
y = df['label']
X = df.drop(['label'],axis=1)

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [24]:
X

,additional_fare,duration,meter_waiting,pickup_time,fare,dist
0,10.5,834.0,56.0,0.20,270.32,5.094369
1,10.5,791.0,47.0,0.56,197.85,3.169052
2,10.5,1087.0,80.0,1.08,301.64,6.307375
3,10.5,598.0,271.0,2.27,82.30,0.862217
5,10.5,3407.0,182.0,5.38,1065.02,24.214638
...,...,...,...,...,...,...
17171,10.5,838.0,93.0,22.07,198.26,2.106037
17172,10.5,2151.0,428.0,23.07,581.23,10.871789
17173,10.5,263.0,9.0,23.21,76.20,1.045847
17174,10.5,858.0,115.0,23.39,133.31,2.879981


In [ ]:
from sklearn.model_selection import learning_curve, GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

xgb_clf = xgb.XGBClassifier(tree_method = "exact", predictor = "cpu_predictor", verbosity = 2)

# Create parameter grid
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.5, 1, 2],
               "max_depth": [2, 7, 10],
               "colsample_bytree": [0.3, 0.8, 1.0],
               "subsample": [0.2, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 4],
               "min_child_weight": [1, 3, 5],
               "n_estimators": [100, 1500]}

from sklearn.model_selection import RandomizedSearchCV
# Create RandomizedSearchCV Object
xgb_rscv = GridSearchCV(xgb_clf, param_grid = parameters, scoring = "f1_micro",
                             cv = 5, verbose = 3, n_jobs=6 )

# Fit the model
model_xgboost = xgb_rscv.fit(X, y)

# Model best estimators
print("Learning Rate: ", model_xgboost.best_estimator_.get_params()["learning_rate"])
print("Gamma: ", model_xgboost.best_estimator_.get_params()["gamma"])
print("Max Depth: ", model_xgboost.best_estimator_.get_params()["max_depth"])
print("Subsample: ", model_xgboost.best_estimator_.get_params()["subsample"])
print("Max Features at Split: ", model_xgboost.best_estimator_.get_params()["colsample_bytree"])
print("Alpha: ", model_xgboost.best_estimator_.get_params()["reg_alpha"])
print("Lamda: ", model_xgboost.best_estimator_.get_params()["reg_lambda"])
print("Minimum Sum of the Instance Weight Hessian to Make a Child: ",
      model_xgboost.best_estimator_.get_params()["min_child_weight"])
print("Number of Trees: ", model_xgboost.best_estimator_.get_params()["n_estimators"])

Fitting 5 folds for each of 7776 candidates, totalling 38880 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 500 tasks      | elapsed:  5.0min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  8.9min
[Parallel(n_jobs=6)]: Done 1140 tasks      | elapsed: 14.9min
[Parallel(n_jobs=6)]: Done 1556 tasks      | elapsed: 19.8min
[Parallel(n_jobs=6)]: Done 2036 tasks      | elapsed: 27.3min
[Parallel(n_jobs=6)]: Done 2580 tasks      | elapsed: 32.7min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed: 40.0min
[Parallel(n_jobs=6)]: Done 3860 tasks      | elapsed: 47.9min
[Parallel(n_jobs=6)]: Done 4596 tasks      | elapsed: 58.0min
[Parallel(n_jobs=6)]: Done 5396 tasks      | elapsed: 69.9min
[Parallel(n_jobs=6)]: Done 6260 tasks      | elapsed: 78.7min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 89.4min


In [30]:
pred=xgb_rscv.predict(X_test)
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy for XgBoost on CV data: ",f1_score(y_test,pred))

Accuracy for XgBoost on CV data:  0.9736607710904694


In [27]:
X

,additional_fare,duration,meter_waiting,pickup_time,fare,dist
0,10.5,834.0,56.0,0.20,270.32,5.094369
1,10.5,791.0,47.0,0.56,197.85,3.169052
2,10.5,1087.0,80.0,1.08,301.64,6.307375
3,10.5,598.0,271.0,2.27,82.30,0.862217
5,10.5,3407.0,182.0,5.38,1065.02,24.214638
...,...,...,...,...,...,...
17171,10.5,838.0,93.0,22.07,198.26,2.106037
17172,10.5,2151.0,428.0,23.07,581.23,10.871789
17173,10.5,263.0,9.0,23.21,76.20,1.045847
17174,10.5,858.0,115.0,23.39,133.31,2.879981


In [28]:
df_test = pd.read_csv('test.csv')
df_test['pick_lat'] = deg_to_rad(df_test['pick_lat'])
df_test['pick_lon'] = deg_to_rad(df_test['pick_lon'])
df_test['drop_lat'] = deg_to_rad(df_test['drop_lat'])
df_test['drop_lon'] = deg_to_rad(df_test['drop_lon'])
dlon = df_test['drop_lon'] - df_test['pick_lon']
dlat = df_test['drop_lat'] - df_test['pick_lat']
df_test['dist'] = np.sin(dlat/2)**2 + np.cos(df_test['pick_lat'])*np.cos(df_test['drop_lat'])*np.sin(dlon/2)**2
df_test['dist'] = 2*np.arctan2(df_test['dist']**0.5,(1-df_test['dist'])**0.5)
R = 6373.0
df_test['dist'] = R*df_test['dist']
df_test['pickup_time'] = [float((d.split(' ')[1]).split(':')[0]+'.'+(d.split(' ')[1]).split(':')[1]) for d in df_test['pickup_time']]
df_test =df_test.drop(['meter_waiting_fare', 'meter_waiting_till_pickup','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1)
df_new = df_test[:]
df_test = df_test.drop(['tripid'], axis = 1)
X_2 = df_test
X_2

,additional_fare,duration,meter_waiting,pickup_time,fare,dist
0,10.5,924,42,0.38,289.27,6.707807
1,10.5,4249,20,1.02,1912.70,41.571559
2,10.5,1552,255,5.02,394.00,5.918535
3,10.5,462,16,5.30,154.32,3.302798
4,10.5,814,392,7.00,147.47,2.589354
...,...,...,...,...,...,...
8571,10.5,1723,429,21.28,388.48,3.935507
8572,10.5,1378,80,21.59,379.85,7.519792
8573,10.5,418,56,22.02,112.79,2.057870
8574,10.5,1604,548,22.07,248.46,3.902113


In [29]:
y_ran_pred=xg.predict(X_2)
df_new['prediction'] = y_ran_pred

NameError: name 'xg' is not defined

In [ ]:
df_new = df_new.drop(['pickup_time','additional_fare','duration','meter_waiting','fare','dist'], axis = 1) 

In [ ]:
df_new.to_csv("prediction_results_xgb.csv",index=False)

In [ ]:
df_new

In [ ]:
df_new['prediction'].value_counts()